In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

DB Password: ········


In [2]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [3]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [4]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    try:
        Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    except:
        print("DataFrame is empty. Condition does not fit to any record!")
        raise SystemExit(0)
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        for mrn in cohort_indexes: 
            x = get_mrn_has_certain_condition_WINDOW(mrn,cohort_condition,df_mrn.loc[mrn]["HT_Onset"]-gap_in_days)
            a = [mrn, x]
            cohort_mrn_diagnosis.append(a)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [5]:
case = df_to_cohort(Case_filtered_15_540)
control = df_to_cohort(Control_filtered_15_540)

In [6]:
Med_CV_condition = (Drug('%Lisinopril%') | 
                    Drug('%Hydrochlorothiazide%')|
                    Drug('%Metoprolol tartrate%')|
                    Drug('%Metoprolol succinate%')|
                    Drug('%Amlodipine besylate%')|
                    Drug('%Atenolol%')|
                    Drug('%Losartan potassium%')|
                    Drug('%Enalapril maleate%')
                    )


In [7]:
##### CASES ######

In [10]:
Med_CV_EVER = get_has_certain_condition(Med_CV_condition, Case_filtered_15_540, "Med_CV", 180 , "EVER", "Case")

Fetching data for Drug (...)


In [11]:
Med_CV_EVER.loc[Med_CV_EVER['has_condition_Med_CV'] == 1]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_CV
medical_record_number,,,,,,,,,,,,,
102832898,555,23045.0,23051.0,NaN,23045.0,Age_BP_condition,22429,616.0,1944,Ba,Baptist,Female,1.0
1034061996,85,27074.0,27031.0,NaN,27031.0,Age_ICD_condition,26202,829.0,1942,Other,Catholic,Female,1.0
1039156031,89,26224.0,NaN,NaN,26224.0,Age_BP_condition,24948,1276.0,1946,Unknown,Christian,Female,1.0
1053244405,136,23046.0,23121.0,NaN,23046.0,Age_BP_condition,17352,5694.0,1952,Other,Catholic,Female,1.0
1054296935,371,18047.0,16223.0,NaN,16223.0,Age_ICD_condition,14659,1564.0,1960,Other,Catholic,Female,1.0
1126813698,48,NaN,25193.0,NaN,25193.0,Age_ICD_condition,24445,748.0,1943,Other,,Female,1.0
1127925844,624,22721.0,22330.0,NaN,22330.0,Age_ICD_condition,11791,10539.0,1952,White,PT Declined,Male,1.0
1162265011,486,18107.0,18146.0,NaN,18107.0,Age_BP_condition,16967,1140.0,1955,Black Or African-American,Baptist,Female,1.0
1169184415,301,14291.0,NaN,NaN,14291.0,Age_BP_condition,9014,5277.0,1976,Other,Catholic,Female,1.0


In [12]:
Med_CV_COUNT = get_has_certain_condition(Med_CV_condition, Case_filtered_15_540, "Med_CV", 180 , "COUNT", "Case")

Fetching data for Drug (...)


In [13]:
Med_CV_COUNT.loc[Med_CV_COUNT['number_of_occurences_Med_CV'] != 0]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_Med_CV
medical_record_number,,,,,,,,,,,,,
102832898,555,23045.0,23051.0,NaN,23045.0,Age_BP_condition,22429,616.0,1944,Ba,Baptist,Female,3.0
1034061996,85,27074.0,27031.0,NaN,27031.0,Age_ICD_condition,26202,829.0,1942,Other,Catholic,Female,1.0
1039156031,89,26224.0,NaN,NaN,26224.0,Age_BP_condition,24948,1276.0,1946,Unknown,Christian,Female,3.0
1053244405,136,23046.0,23121.0,NaN,23046.0,Age_BP_condition,17352,5694.0,1952,Other,Catholic,Female,2.0
1054296935,371,18047.0,16223.0,NaN,16223.0,Age_ICD_condition,14659,1564.0,1960,Other,Catholic,Female,2.0
1126813698,48,NaN,25193.0,NaN,25193.0,Age_ICD_condition,24445,748.0,1943,Other,,Female,1.0
1127925844,624,22721.0,22330.0,NaN,22330.0,Age_ICD_condition,11791,10539.0,1952,White,PT Declined,Male,1.0
1162265011,486,18107.0,18146.0,NaN,18107.0,Age_BP_condition,16967,1140.0,1955,Black Or African-American,Baptist,Female,1.0
1169184415,301,14291.0,NaN,NaN,14291.0,Age_BP_condition,9014,5277.0,1976,Other,Catholic,Female,1.0


In [14]:
Med_CV = Med_CV_EVER.merge(Med_CV_COUNT['number_of_occurences_Med_CV'], left_index= True, right_index = True)

In [15]:
Med_CV

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_CV,number_of_occurences_Med_CV
medical_record_number,,,,,,,,,,,,,,
100067499,82,24103.0,NaN,24106.0,24103.0,Age_BP_condition,21049,3054.0,1951,White,Other,Male,0.0,0.0
100091035,154,11369.0,12938.0,12881.0,11369.0,Age_BP_condition,9141,2228.0,1978,Ba,Other,Male,0.0,0.0
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,0.0,0.0
100423963,20,NaN,23584.0,23584.0,23584.0,Age_ICD_condition,19827,3757.0,1945,Other,Pt Declined,Male,0.0,0.0
1005188482,279,20636.0,20514.0,20514.0,20514.0,Age_ICD_condition,19884,630.0,1958,Ba,Jewish,Male,0.0,0.0
1006954100,28,15242.0,15536.0,15536.0,15242.0,Age_BP_condition,13828,1414.0,1973,Unknown,Catholic,Female,0.0,0.0
1006987597,72,27618.0,27618.0,27618.0,27618.0,Age_BP_condition,26706,912.0,1941,White,Catholic,Male,0.0,0.0
1007749798,93,NaN,27233.0,27233.0,27233.0,Age_ICD_condition,25235,1998.0,1939,White,Catholic,Female,0.0,0.0
1007977434,221,25355.0,24347.0,24347.0,24347.0,Age_ICD_condition,23605,742.0,1943,White,Unknown,Male,0.0,0.0


In [16]:
Med_CV.to_parquet('Med_CV_Case.parquet')

In [17]:
##### CONTROLS######

In [8]:
Med_CV_EVER_Control = get_has_certain_condition(Med_CV_condition, Control_filtered_15_540, "Med_CV", 180 , "EVER", "Control")


Fetching data for Drug (...)


In [9]:
Med_CV_EVER_Control.loc[Med_CV_EVER_Control['has_condition_Med_CV'] == 1]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_CV
medical_record_number,,,,,,,,,
1930785744,6057,41,1812,4245,2000,Black Or African-American,Other,Female,1.0
2082999752,28291,223,24098,4193,1936,White,Jewish,Male,1.0
2386314908,31407,42,29504,1903,1929,Black Or African-American,Unknown,Female,1.0
255638265,17492,23,15801,1691,1969,Black Or African-American,Christian,Female,1.0
2688587780,23439,15,22159,1280,1954,White,Unknown,Female,1.0
3042746768,21580,69,19436,2144,1959,White,,Male,1.0
3178155540,23319,34,20952,2367,1954,White,None,Female,1.0
3259891192,30178,18,28503,1675,1933,Unknown,,Male,1.0
3289795948,21567,34,19973,1594,1953,White,Jewish,Female,1.0


In [11]:
Med_CV_COUNT_Control = get_has_certain_condition(Med_CV_condition, Control_filtered_15_540, "Med_CV", 180 , "COUNT", "Control")

Fetching data for Drug (...)


In [12]:
Med_CV_COUNT_Control.loc[Med_CV_COUNT_Control['number_of_occurences_Med_CV'] != 0]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_Med_CV
medical_record_number,,,,,,,,,
1930785744,6057,41,1812,4245,2000,Black Or African-American,Other,Female,3.0
2082999752,28291,223,24098,4193,1936,White,Jewish,Male,6.0
2386314908,31407,42,29504,1903,1929,Black Or African-American,Unknown,Female,1.0
255638265,17492,23,15801,1691,1969,Black Or African-American,Christian,Female,1.0
2688587780,23439,15,22159,1280,1954,White,Unknown,Female,1.0
3042746768,21580,69,19436,2144,1959,White,,Male,1.0
3178155540,23319,34,20952,2367,1954,White,None,Female,2.0
3259891192,30178,18,28503,1675,1933,Unknown,,Male,1.0
3289795948,21567,34,19973,1594,1953,White,Jewish,Female,1.0


In [13]:
Med_CV_control = Med_CV_EVER_Control.merge(Med_CV_COUNT_Control['number_of_occurences_Med_CV'], left_index = True, right_index = True)


In [14]:
Med_CV_control

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_Med_CV,number_of_occurences_Med_CV
medical_record_number,,,,,,,,,,
100002558,21414,15,19951,1463,1959,White,Other,Female,0.0,0.0
100002884,23929,44,18526,5403,1952,Unknown,,Male,0.0,0.0
1000068212,10116,29,9228,888,1990,White,Unknown,Female,0.0,0.0
1000083464,25630,20,23026,2604,1948,White,Catholic,Male,0.0,0.0
1000119948,17087,163,15757,1330,1971,African-American,None,Male,0.0,0.0
1000165222,17506,627,12215,5291,1970,Other,Catholic,Female,0.0,0.0
1000199143,26941,141,22805,4136,1944,White,Jewish,Female,0.0,0.0
1000205924,14867,108,10618,4249,1976,Unknown,Unknown,Female,0.0,0.0
1000212584,19220,15,16818,2402,1965,White,,Male,0.0,0.0


In [15]:
Med_CV_control.to_parquet('Med_CV_control.parquet')